In [3]:
import numpy as np
from __future__ import division

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import matplotlib.pyplot as plt
from sklearn import svm,preprocessing
import pandas as pd
from matplotlib import style
import statistics

In [6]:
FEATURES = ["DE Ratio",
                      'Trailing P/E',
                      'Price/Sales',
                      'Price/Book',
                      'Profit Margin',
                      'Operating Margin',
                      'Return on Assets',
                      'Return on Equity',
                      'Revenue Per Share',
                      'Market Cap',
                        'Enterprise Value',
                        'Forward P/E',
                        'PEG Ratio',
                        'Enterprise Value/Revenue',
                        'Enterprise Value/EBITDA',
                        'Revenue',
                        'Gross Profit',
                        'EBITDA',
                        'Net Income Avl to Common ',
                        'Diluted EPS',
                        'Earnings Growth',
                        'Revenue Growth',
                        'Total Cash',
                        'Total Cash Per Share',
                        'Total Debt',
                        'Current Ratio',
                        'Book Value Per Share',
                        'Cash Flow',
                        'Beta',
                        'Held by Insiders',
                        'Held by Institutions',
                        'Shares Short (as of',
                        'Short Ratio',
                        'Short % of Float',
                        'Shares Short (prior ']

style.use("ggplot")
how_much_better = 5

In [7]:
def Status_Calc(stock,sp500):
    difference = stock - sp500
    
    if difference > how_much_better:
        return 1
    else:
        return 0

In [8]:
def Build_Data_Set():
    
    #data_df = pd.DataFrame.from_csv("key_stats.csv")
    data_df = pd.DataFrame.from_csv("key_stats_acc_perf_NO_NA.csv")
    data_df = data_df.reindex(np.random.permutation(data_df.index))
    data_df = data_df.replace("NaN",0).replace("N/A",0)
    
    data_df["Status2"] = list(map(Status_Calc,data_df["stock_p_change"],
                                 data_df["sp500_p_change"]))
    
    
    X = np.array(data_df[FEATURES].values)
    y = (data_df["Status2"]
        .replace("underperform",0)
        .replace("outperform",1)
        .values.tolist())
    X = preprocessing.scale(X)
    Z = np.array(data_df[["stock_p_change","sp500_p_change"]])
    
    return X,y,Z

In [9]:
def Analysis():    
    test_size = 1
    invest_amount = 10000
    total_invests = 0
    if_market = 0
    if_strat = 0  
    
    X,y,Z = Build_Data_Set()
    print len(X)
    print "X"
    print X
    clf = svm.SVC(kernel = "linear",C=1.0)
    clf.fit(X[:-test_size],y[:-test_size])
    
    correct_count = 0
    
    for x in range(1,test_size+1):
        if clf.predict(X[-x])[0] == y[-x]:
            correct_count+=1
            
        if clf.predict(X[-x])[0] == 1:
            invest_return = invest_amount + (invest_amount*(Z[-x][0]/100))
            market_return = invest_amount + (invest_amount*(Z[-x][1]/100))
            
            total_invests += 1
            if_market += market_return
            if_strat += invest_return
            
    data_df = pd.DataFrame.from_csv("forward_sample_NO_NA.csv")
    data_df = data_df.replace("N/A",0).replace("NaN",0)
    X = np.array(data_df[FEATURES].values)
    X = preprocessing.scale(X)
    Z = data_df["Ticker"].values.tolist()
    
    invest_list = []
    
    for i in xrange(len(X)):
        p = clf.predict(X[i])[0]
        if p==1:
            print (Z[i])
            invest_list.append(Z[i])
            
    print (len(invest_list))
    print invest_list
    

In [10]:
Analysis()

9236
X
[[-0.36835799  1.092487    0.00400417 ..., -0.51140054 -0.16976418
  -0.06417012]
 [-0.36753031  0.91282441  0.1352234  ..., -0.79911379 -0.35860091
   0.51670473]
 [-0.36835799  0.13726468  0.15720429 ..., -0.48476043 -0.2080419
  -0.44053157]
 ..., 
 [-0.2811328   0.46998038 -0.01398019 ..., -0.56468077  0.41716078
  -0.56338163]
 [-0.36835799  0.20951818  0.24113131 ...,  0.05923073 -0.50660807
  -0.62906736]
 [-0.34602325 -0.02394608 -0.15785508 ...,  1.61954229 -0.256527
  -0.28395101]]


KeyboardInterrupt: 